# Segmentació d’Imatges amb U-Net

En aquesta pràctica treballarem amb U-Net, una de les arquitectures més utilitzades en tasques de segmentació d'imatges. L'objectiu principal és consolidar els coneixements teòrics adquirits, implementant i experimentant amb aquesta xarxa utilitzant un conjunt de dades artificial i simple.

La segmentació d'imatges és una tasca fonamental en visió per computador, ja que permet classificar cada píxel d'una imatge en una o més categories. En aquesta pràctica, el nostre objectiu serà entrenar una xarxa U-Net perquè pugui identificar regions específiques dins d’imatges generades artificialment. Aquest enfocament simplificat permetrà concentrar-nos en els aspectes clau de la implementació i el funcionament del model, sense les complicacions que podrien sorgir amb conjunts de dades més complexos.

El conjunt de dades que utilitzarem estarà format per imatges sintètiques que contenen formes geomètriques bàsiques (com cercles, quadrats o creus), amb les seves corresponents màscares que indiquen les àrees que cal segmentar. Això ens permetrà obtenir resultats visuals ràpidament i entendre millor com la xarxa aprèn a identificar patrons específics.

L'arquitectura que emprarem es pot utilitzar, per problemes més complexes, però amb les limitacions temporals que tenim no es recomenable.

In [7]:
from collections import OrderedDict
from glob import glob

import cv2
import matplotlib.pyplot as plt
import numpy as np
import pylab as pl
import torch
import torch.nn as nn
import torch.optim as optim
import argparse
from IPython import display
from torch.utils.data import Dataset
from torchvision import transforms
from tqdm.auto import tqdm


## Dataset

Emprarem un dataset sintètic. Aquest originalment s'emprava per dur a terme tasques de Intel·ligència Artificial Explicable (XAI). Nosaltres ho emprarem per dur a terme una tasca de segmentació. Podeu trobar el dataset complet al següent [enllaç](https://github.com/miquelmn/aixi-dataset/releases/tag/1.5.0).

![Imatges dataset](https://ars.els-cdn.com/content/image/1-s2.0-S0004370224001152-gr001.jpg)

L'objectiu amb aquest dataset es segmentar del fons les formes geomètriques, emprant una U-Net. Nosaltres i a causa de les limitacions temporals que tenim i de recursos emprarem una versió [reduida](https://github.com/miquelmn/aa_2526/releases/tag/pr9).

In [4]:
!unzip mini_txuxi.zip -d /content/

Archive:  mini_txuxi.zip
   creating: /content/mini/
   creating: /content/mini/train/
   creating: /content/mini/train/image/
  inflating: /content/mini/train/image/00000.png  
  inflating: /content/mini/train/image/00001.png  
  inflating: /content/mini/train/image/00002.png  
  inflating: /content/mini/train/image/00003.png  
  inflating: /content/mini/train/image/00004.png  
  inflating: /content/mini/train/image/00005.png  
  inflating: /content/mini/train/image/00006.png  
  inflating: /content/mini/train/image/00007.png  
  inflating: /content/mini/train/image/00008.png  
  inflating: /content/mini/train/image/00009.png  
  inflating: /content/mini/train/image/00010.png  
  inflating: /content/mini/train/image/00011.png  
  inflating: /content/mini/train/image/00012.png  
  inflating: /content/mini/train/image/00013.png  
  inflating: /content/mini/train/image/00014.png  
  inflating: /content/mini/train/image/00015.png  
  inflating: /content/mini/train/image/00016.png  
  infl

# Dataset de segmentació

A l’hora de treballar amb tasques de segmentació d’imatges, és essencial tenir un conjunt de dades que inclogui tant les imatges d’entrada com les màscares que representen les etiquetes de segmentació. Un dataset per segmentació ha de contenir:

1. Imatges d’entrada: Aquestes són les imatges que el model utilitzarà per aprendre. Poden estar en formats com JPEG o PNG.
2. Màscares de segmentació: Cada màscara és una imatge on cada píxel té un valor que representa la classe a la qual pertany (per exemple, 0 per fons, 1 per objecte). Les màscares han de tenir la mateixa mida que les imatges d’entrada.

In [3]:
class Formes(Dataset):
    """TXUXI segmentation dataset."""
    def __init__(self, img_files, label_files, size=(256,256), multiclass=False, n_classes=1):
        assert len(img_files) == len(label_files)
        self.img_files = img_files
        self.label_files = label_files
        self.size = size
        self.multiclass = multiclass
        self.n_classes = n_classes

    def __len__(self):
        return len(self.img_files)

    def __getitem__(self, idx):
        # Leer imagen RGB
        img = cv2.imread(self.img_files[idx], cv2.IMREAD_COLOR)
        if img is None:
            raise FileNotFoundError(self.img_files[idx])
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        # Leer máscara (en escala de grises o con etiquetas)
        mask = cv2.imread(self.label_files[idx], cv2.IMREAD_UNCHANGED)
        if mask is None:
            raise FileNotFoundError(self.label_files[idx])

        # Redimensionar
        h, w = self.size
        img = cv2.resize(img, (w, h), interpolation=cv2.INTER_LINEAR)
        # Si máscara tiene canales (p.ej PNG RGBA), tomar solo primer canal o convertir adecuadamente
        if mask.ndim == 3:
            # si máscara tiene 3 canales y representa etiquetas en colores, conviene convertir a grises
            mask = cv2.cvtColor(mask, cv2.COLOR_BGR2GRAY)
        mask = cv2.resize(mask, (w, h), interpolation=cv2.INTER_NEAREST)

        # Normalizar imagen a [0,1] y permutar a C,H,W
        img = img.astype(np.float32) / 255.0
        img = np.transpose(img, (2,0,1))  # [C,H,W]

        if self.multiclass:
            # Esperamos que mask contenga valores 0..n_classes-1
            mask_tensor = torch.from_numpy(mask.astype(np.int64))  # [H,W], dtype long
            return torch.from_numpy(img), mask_tensor
        else:
            # Binarizar: >127 => 1
            mask_bin = (mask > 127).astype(np.float32)
            mask_bin = np.expand_dims(mask_bin, axis=0)  # [1,H,W]
            return torch.from_numpy(img), torch.from_numpy(mask_bin)
    #TODO

Per instanciar aquest tipus de dataset es similar a com ho hem fet fins ara. **Per tal de simplificar l'entrenament només emprarem 500 mostres per l'entrenament i 200 de test**.

In [9]:
PATH_DADES = "./mini"  # POSA EL TEU!
# Dades entrenament
path_train = f"{PATH_DADES}/train"

img_files = sorted(glob(path_train + "/image/*.png"))
label_files = sorted(glob(path_train + "/mask/*.png"))
img_files = img_files[:500]
label_files = label_files[:500]

print("total training images", len(img_files))

# Dades validacio

path_val = f"{PATH_DADES}/val"
img_files_val = sorted(glob(path_val + "/image/*.png"))
label_files_val = sorted(glob(path_val + "/mask/*.png"))
img_files_val = img_files_val[:200]
label_files_val = label_files_val[:200]

print("total test images", len(img_files_val))

total training images 500
total test images 200


In [10]:
train_batch_size = 4
test_batch_size = 4

# Definim una seqüència (composició) de transformacions
transform = transforms.Compose([
    transforms.ToTensor(),
    ## TODO: Put if necessary
])

train_data = Formes(img_files, label_files, transform)
val_data = Formes(img_files_val, label_files_val, transform)

train_loader = torch.utils.data.DataLoader(train_data, train_batch_size)
val_loader = torch.utils.data.DataLoader(val_data, test_batch_size)

## Entrenament
L’entrenament d’una xarxa U-Net consisteix a ensenyar al model a predir les màscares de segmentació corresponents a les imatges d’entrada. Aquest procés implica ajustar els pesos de la xarxa per minimitzar l’error entre les prediccions i les etiquetes reals (màscares). En aquesta secció, configurarem tot el necessari per entrenar el model amb el conjunt de dades generat prèviament.

Hi ha tot un conjunt de peculiaritats que fan que aquest entrenament difereixi respecte els vists fins ara:
- **Funció de pèrdua**. El Dice Coefficient és una mesura d’avaluació utilitzada en segmentació d’imatges per comparar la superposició entre la màscara predita i la màscara real. El seu valor oscil·la entre 0 (cap coincidència) i 1 (coincidència perfecta), i es calcula com el doble de la intersecció entre les dues màscares dividit per la seva suma total. També es pot emprar ``BCE``.
- **Sortida de la xarxa**. El tipus de problema determina el nombre de canals de sortida de la U-Net: per segmentació binària, la sortida és un únic canal amb valors que representen la probabilitat de pertànyer a la classe positiva (fons o objecte). En canvi, per segmentació multiclasse, la sortida té un canal per a cada classe i els valors representen la probabilitat de cada píxel de pertànyer a cadascuna de les classes.
- **Funcions d'activació**. Les funcions d'activació a la sortida d'una U-Net depenen del tipus de segmentació: en segmentació binària, s'utilitza una sigmoid per comprimir els valors entre 0 i 1, representant la probabilitat de pertànyer a la classe positiva. En segmentació, **sense superposició**, multiclasse, s'aplica una softmax per convertir els valors de cada canal en probabilitats normalitzades, assegurant que la suma sigui 1 per píxel.

### El model

In [11]:
class UNet(nn.Module):
    def __init__(self, in_channels=3, out_channels=1, init_features=32):
        super().__init__()
        f = init_features
        # Encoder
        self.enc1 = DoubleConv(in_channels, f)
        self.pool1 = nn.MaxPool2d(2)
        self.enc2 = DoubleConv(f, f*2)
        self.pool2 = nn.MaxPool2d(2)
        self.enc3 = DoubleConv(f*2, f*4)
        self.pool3 = nn.MaxPool2d(2)
        self.enc4 = DoubleConv(f*4, f*8)
        self.pool4 = nn.MaxPool2d(2)
        # Bottleneck
        self.bottleneck = DoubleConv(f*8, f*16)
        # Decoder
        self.up4 = nn.ConvTranspose2d(f*16, f*8, kernel_size=2, stride=2)
        self.dec4 = DoubleConv(f*16, f*8)
        self.up3 = nn.ConvTranspose2d(f*8, f*4, kernel_size=2, stride=2)
        self.dec3 = DoubleConv(f*8, f*4)
        self.up2 = nn.ConvTranspose2d(f*4, f*2, kernel_size=2, stride=2)
        self.dec2 = DoubleConv(f*4, f*2)
        self.up1 = nn.ConvTranspose2d(f*2, f, kernel_size=2, stride=2)
        self.dec1 = DoubleConv(f*2, f)
        # Final
        self.final_conv = nn.Conv2d(f, out_channels, kernel_size=1)

    def forward(self, x):
        e1 = self.enc1(x)
        p1 = self.pool1(e1)
        e2 = self.enc2(p1)
        p2 = self.pool2(e2)
        e3 = self.enc3(p2)
        p3 = self.pool3(e3)
        e4 = self.enc4(p3)
        p4 = self.pool4(e4)
        b = self.bottleneck(p4)
        u4 = self.up4(b)
        d4 = torch.cat([u4, e4], dim=1)
        d4 = self.dec4(d4)
        u3 = self.up3(d4)
        d3 = torch.cat([u3, e3], dim=1)
        d3 = self.dec3(d3)
        u2 = self.up2(d3)
        d2 = torch.cat([u2, e2], dim=1)
        d2 = self.dec2(d2)
        u1 = self.up1(d2)
        d1 = torch.cat([u1, e1], dim=1)
        d1 = self.dec1(d1)
        out = self.final_conv(d1)
        return out


## La funció de pèrdua

Per fer tasques de segmentació, una de les funcions de pèrdua que podem emprar és el _Diceloss_ (intersecció vs unió):  El coeficient de _Dice_ s'utilitza habitualment en tasques de segmentació d'imatges com a mesura de la superposició entre les màscares de segmentació entre la predicció i el _ground truth_. El  _Diceloss_, és el complementari del coeficient de _Dice_, es pot utilitzar com a funció de pèrdua per entrenar models per a tasques de segmentació.

Dice Coefficient $= 2 \times \frac{|X \cap Y|}{|X| + |Y|}$



On:

- $X$ és la màscara de segmentació prevista.
- $Y$ és la màscara de segmentació de la veritat del sòl.
- $∣⋅∣$ denota la cardinalitat o el nombre d'elements d'un conjunt.

In [12]:
class DiceLoss(nn.Module):

    def __init__(self):
        super(DiceLoss, self).__init__()
        self.smooth = 0.0

    def forward(self, y_pred, y_true):
        assert y_pred.size() == y_true.size()
        y_pred = y_pred[:, 0].contiguous().view(-1)
        y_true = y_true[:, 0].contiguous().view(-1)
        intersection = (y_pred * y_true).sum()
        dsc = (2. * intersection + self.smooth) / (
                y_pred.sum() + y_true.sum() + self.smooth
        )
        return 1. - dsc

### Bucle d'entrenament

El bucle d'entrenament és un poc diferent al vist fins ara. En particular, en aquests moments els gràfics es van actualitzant a mesura que aprèn i així podem tenir un idea de cm va l'entrenament.

In [15]:
def train_one_epoch(model, loader, device, optimizer, multiclass=False, n_classes=1, loss_alpha=0.5):
    model.train()
    running_loss = 0.0
    running_metric = 0.0
    total = 0
    for imgs, masks in tqdm(loader, desc="Train", leave=False):
        imgs = imgs.to(device).float()
        if multiclass:
            masks = masks.to(device).long()  # [B,H,W]
        else:
            masks = masks.to(device).float()  # [B,1,H,W]

        optimizer.zero_grad()
        logits = model(imgs)

        if multiclass:
            # logits [B,C,H,W], masks [B,H,W]
            loss = nn.CrossEntropyLoss()(logits, masks)
        else:
            loss = combined_bce_dice(logits, masks, alpha=loss_alpha)

        loss.backward()
        optimizer.step()

        running_loss += loss.item() * imgs.size(0)
        # métricas
        with torch.no_grad():
            if multiclass:
                metric = mean_iou_multiclass(logits, masks, n_classes)
            else:
                metric = float(dice_from_logits_binary(logits, masks))
            running_metric += metric * imgs.size(0)
        total += imgs.size(0)

    return running_loss / total, running_metric / total


IndentationError: expected an indented block after function definition on line 37 (ipython-input-1417921994.py, line 38)

# Tasques a fer

1. Implementa la U-Net per detectar les formes geomètriques.
2. Empra com a funció de pèrdua tant la ``Dice`` com la ``BCE``, quina diferència hi ha?
3. Fer un nou entrenament per segmentar de forma separada els diferents tipus de figura. Has d'adaptar el model. Quins canvis faries?